In [1]:
import sys

sys.path.insert(0, "..")
import torch
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import cocpit
from cocpit import config as config
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

In [2]:
#  make predictions on each batch of images,
#  concatenate the precitions across batches,
#  and return the wrong predictions
# sourcery skip: use-itertools-product
p = cocpit.predictions.LoaderPredictions()
with torch.no_grad():
    for fold in range(config.KFOLD+1):
        for ((imgs, labels, paths), _) in p.load_val_loader(fold):
            b = cocpit.predictions.BatchPredictions(imgs, p.load_model(fold))
            b.find_max_preds()
            b.top_k_preds(len(config.CLASS_NAMES))
            p.append_batch(b, paths, labels)

p.concatenate_loader_vars()
p.find_wrong_indices()

# val_loader = torch.load(config.VAL_LOADER_SAVENAME)
# with torch.no_grad():
#     for fold in range(config.KFOLD+1):
#         for (imgs, paths) in val_loader:
#             model = p.load_model(fold)
#             b = cocpit.predictions.BatchPredictions(imgs, model)
#             b.find_max_preds()
#             b.top_k_preds(len(config.CLASS_NAMES))
#             p.append_batch(b, paths)
# p.concatenate_loader_vars()
# p.find_wrong_indices()

print("DONE FINDING INCORRECT PREDICTIONS!")                
print(f"There are {len(p.labels)} total images in the dataloader.")
print(f"There are {len(p.wrong_idx)} wrong predictions to check in the dataloader!")


DONE FINDING INCORRECT PREDICTIONS!
There are 5187 total images in the dataloader.
There are 354 wrong predictions to check in the dataloader!


In [3]:
# to only look between specific categories run this cell
label_list = dict(zip(cocpit.config.CLASS_NAMES, np.arange(0, len(cocpit.config.CLASS_NAMES))))

# change these two lines to focus on wrong predictions from a specific category
# or box within the confusion matrix
# makes labeling easier focusing on two at a time
human_label = label_list["obstructed"]
model_label = label_list["precipitation"]

p.hone_incorrect_predictions(label_list, human_label, model_label)

28 wrong predictions between obstructed and precipitation


In [4]:
"""
Display a bar chart of model prediction probabilities
A dropdown menu is available to move the image to a different category in the training dataset
If you don't want to move the image and the human labeled correctly, simply click "Next"
"""
gui = cocpit.gui_wrong.GUI(p.paths, p.topk_probs, p.topk_classes, wrong_trunc=p.wrong_trunc, labels=p.labels, precip=None)
gui.interp()
display(ipywidgets.HBox([gui.output, gui.next_btn]))

In [27]:
# clear out the incorrect predictions between a specific scenario for human and model label.
# If this cell is run after all images have been iterated through, run the second 
# to last cell again to hone in on a different set of human and model predictions
p.wrong_trunc=[]